### 절대 모멘텀
1. 파생변수 'STD-YM' 생성 -> 인덱스에서 년-월을 추출하여 대입 
2. 'STD-YM' 별 마지막날의 데이터들을 month_last_df 추가 (단순 행 결합)
3. 전월의 수정 종가 값을 가진 파생변수 생성 
4. 전년도의 수정 종가 값을 가진 파생변수 생성 
5. 전월의 데이터와 전년도의 데이터를 이용하여 거래 내역 생성 
6. 수익율 계산

In [1]:
import pandas as pd
import numpy as np 
from datetime import datetime
import warnings

In [2]:
# 위험 메시지 출력 필터
warnings.filterwarnings('ignore')

In [5]:
df = pd.read_csv("주식data/삼성카드.csv")
df

,날짜,시가,고가,저가,종가,거래량,등락률,종목코드
0,2015-01-02,43650,44150,43500,43750,56764,-1.018100,29780
1,2015-01-05,43050,44000,41250,41300,233274,-5.600000,29780
2,2015-01-06,41000,41050,40050,40150,266237,-2.784504,29780
3,2015-01-07,40250,40600,39700,39950,150131,-0.498132,29780
4,2015-01-08,40050,40950,40050,40750,111397,2.002503,29780
...,...,...,...,...,...,...,...,...
2453,2024-12-23,40000,41550,40000,41450,63282,3.754693,29780
2454,2024-12-24,41450,41600,40400,40900,56586,-1.326900,29780
2455,2024-12-26,40800,41000,40600,40950,54477,0.122249,29780
2456,2024-12-27,40500,40650,39800,39950,58149,-2.442002,29780


In [8]:
if '날짜' in df.columns:
    df.set_index('날짜', inplace=True)

In [9]:
# index를 시계열로 전환
df.index = pd.to_datetime(df.index)

In [10]:
# df에 STD-YM 컬럼을 생성하여 인덱스에서 년도-월 데이터를 추출하여 대입
df['STD-YM'] = df.index.strftime('%Y-%m')

In [11]:
# 빈 데이터프레임에 월말의 데이터들을 행결합 
# STD-YM에서 다음 행과 현재 행이 다르다면
flag = df['STD-YM'] != df['STD-YM'].shift(-1)
df.loc[flag, ]

,시가,고가,저가,종가,거래량,등락률,종목코드,STD-YM
날짜,,,,,,,,
2015-01-30,39250,39250,37500,38200,208219,-0.261097,29780,2015-01
2015-02-27,37250,37650,37100,37500,135607,0.536193,29780,2015-02
2015-03-31,40250,41100,40000,41050,205336,3.140704,29780,2015-03
2015-04-30,41700,42850,41050,41100,289330,-1.674641,29780,2015-04
2015-05-29,41300,42300,41250,41800,157307,-0.357569,29780,2015-05
...,...,...,...,...,...,...,...,...
2024-08-30,44500,44650,43300,43450,53228,-2.359551,29780,2024-08
2024-09-30,41300,41450,40500,41150,69600,1.354680,29780,2024-09
2024-10-31,40350,40400,39600,40400,64846,0.622665,29780,2024-10


In [12]:
# 기준년월의 유니크 값을 따로 생성 
month_list = df['STD-YM'].unique()
month_list

array(['2015-01', '2015-02', '2015-03', '2015-04', '2015-05', '2015-06',
       '2015-07', '2015-08', '2015-09', '2015-10', '2015-11', '2015-12',
       '2016-01', '2016-02', '2016-03', '2016-04', '2016-05', '2016-06',
       '2016-07', '2016-08', '2016-09', '2016-10', '2016-11', '2016-12',
       '2017-01', '2017-02', '2017-03', '2017-04', '2017-05', '2017-06',
       '2017-07', '2017-08', '2017-09', '2017-10', '2017-11', '2017-12',
       '2018-01', '2018-02', '2018-03', '2018-04', '2018-05', '2018-06',
       '2018-07', '2018-08', '2018-09', '2018-10', '2018-11', '2018-12',
       '2019-01', '2019-02', '2019-03', '2019-04', '2019-05', '2019-06',
       '2019-07', '2019-08', '2019-09', '2019-10', '2019-11', '2019-12',
       '2020-01', '2020-02', '2020-03', '2020-04', '2020-05', '2020-06',
       '2020-07', '2020-08', '2020-09', '2020-10', '2020-11', '2020-12',
       '2021-01', '2021-02', '2021-03', '2021-04', '2021-05', '2021-06',
       '2021-07', '2021-08', '2021-09', '2021-10', 

In [13]:
month_last_df = pd.DataFrame()

for month in month_list:
    # flag = df['STD-YM'] == month
    # data = df.loc[flag, ].tail(1)
    data = df.loc[month, ].tail(1)
    month_last_df = pd.concat(
        [month_last_df, data], axis=0
    )

In [15]:
month_last_df = month_last_df[['종가', 'STD-YM']]
month_last_df.head()

,종가,STD-YM
날짜,,
2015-01-30,38200,2015-01
2015-02-27,37500,2015-02
2015-03-31,41050,2015-03
2015-04-30,41100,2015-04
2015-05-29,41800,2015-05


In [17]:
# 전월의 수정종가 파생변수(BF-1M) 생성
month_last_df['BF-1M'] = month_last_df['종가'].shift(1).fillna(0)
# 전년도의 수정종가 파생변수(BF-12M) 생성
month_last_df['BF-12M'] = month_last_df['종가'].shift(12).fillna(0)

In [18]:
month_last_df

,종가,STD-YM,BF-1M,BF-12M
날짜,,,,
2015-01-30,38200,2015-01,0.0,0.0
2015-02-27,37500,2015-02,38200.0,0.0
2015-03-31,41050,2015-03,37500.0,0.0
2015-04-30,41100,2015-04,41050.0,0.0
2015-05-29,41800,2015-05,41100.0,0.0
...,...,...,...,...
2024-08-30,43450,2024-08,42100.0,29950.0
2024-09-30,41150,2024-09,43450.0,30450.0
2024-10-31,40400,2024-10,41150.0,31350.0


- 거래 내역 추가 
    - (전월의 수정주가) / (전년도의 수정주가) - 1 값이 0보다 크고 무한대가 아닌 경우가 매수 타이밍
    - 모멘텀 인덱스가 위의 조건에 부합할 경우 df의 trade 컬럼에 해당 월에 구매 내역 추가 

In [19]:
df['trade'] = ""

# month_last_df의 인덱스를 기준으로 반복문 실행 
for idx in month_last_df.index:
    signal = ""

    # 절대 모멘텀 인덱스를 계산 
    momentum_index = (month_last_df.loc[idx, 'BF-1M'] / 
                    month_last_df.loc[idx, 'BF-12M']) - 1
    # print(momentum_index)
    # break
    # 조건식 : 모멘텀 인덱스가 0보다 크고 무한대가 아닌
    # 참인 경우 데이터 if 조건식 else 거짓인 경우 데이터
    flag = True if (momentum_index > 0) & (momentum_index != np.inf) \
        else False

    if flag:
        signal = 'buy'
    print(f"날짜 : {idx}, 모멘텀 인덱스 : {momentum_index}, signal : {signal}")
    df.loc[idx:, 'trade'] = signal

날짜 : 2015-01-30 00:00:00, 모멘텀 인덱스 : nan, signal : 
날짜 : 2015-02-27 00:00:00, 모멘텀 인덱스 : inf, signal : 
날짜 : 2015-03-31 00:00:00, 모멘텀 인덱스 : inf, signal : 
날짜 : 2015-04-30 00:00:00, 모멘텀 인덱스 : inf, signal : 
날짜 : 2015-05-29 00:00:00, 모멘텀 인덱스 : inf, signal : 
날짜 : 2015-06-30 00:00:00, 모멘텀 인덱스 : inf, signal : 
날짜 : 2015-07-31 00:00:00, 모멘텀 인덱스 : inf, signal : 
날짜 : 2015-08-31 00:00:00, 모멘텀 인덱스 : inf, signal : 
날짜 : 2015-09-30 00:00:00, 모멘텀 인덱스 : inf, signal : 
날짜 : 2015-10-30 00:00:00, 모멘텀 인덱스 : inf, signal : 
날짜 : 2015-11-30 00:00:00, 모멘텀 인덱스 : inf, signal : 
날짜 : 2015-12-30 00:00:00, 모멘텀 인덱스 : inf, signal : 
날짜 : 2016-01-29 00:00:00, 모멘텀 인덱스 : -0.19240837696335078, signal : 
날짜 : 2016-02-29 00:00:00, 모멘텀 인덱스 : 0.014666666666666606, signal : buy
날짜 : 2016-03-31 00:00:00, 모멘텀 인덱스 : -0.1303288672350792, signal : 
날짜 : 2016-04-29 00:00:00, 모멘텀 인덱스 : -0.08394160583941601, signal : 
날짜 : 2016-05-31 00:00:00, 모멘텀 인덱스 : -0.0717703349282297, signal : 
날짜 : 2016-06-30 00:00:00, 모멘텀 인덱스 : 0.036486486

In [20]:
df['trade'].value_counts()

trade
       1514
buy     944
Name: count, dtype: int64

In [22]:
# 수익율 계산
# 매수 일자 : 전날의 trade가 ""이고 오늘의 trade "buy"
# 매도 일자 : 전날의 trade가 "buy"이고 오늘의 trade ""

df['rtn'] = 1

for idx in df.index:
    # 매수의 조건식 
    if (df.shift().loc[idx, 'trade'] == "") & \
        (df.loc[idx, 'trade'] == "buy"):
        buy = df.loc[idx, '종가']
        print(f"매수일 : {idx}, 매수가 : {buy}")
    # 매도의 조건식
    elif (df.shift().loc[idx, 'trade'] == 'buy') & \
        (df.loc[idx, 'trade'] == ""):
        sell = df.loc[idx, '종가']
        rtn = sell / buy
        df.loc[idx, 'rtn'] = rtn
        print(f"매도일 : {idx}, 매도가 : {sell}, 수익율 : {rtn}")

매수일 : 2016-02-29 00:00:00, 매수가 : 35700
매도일 : 2016-03-31 00:00:00, 매도가 : 37650, 수익율 : 1.0546218487394958
매수일 : 2016-06-30 00:00:00, 매수가 : 41500
매도일 : 2017-07-31 00:00:00, 매도가 : 39700, 수익율 : 0.9566265060240964
매수일 : 2018-01-31 00:00:00, 매수가 : 38950
매도일 : 2018-02-28 00:00:00, 매도가 : 36300, 수익율 : 0.9319640564826701
매수일 : 2018-10-31 00:00:00, 매수가 : 33650
매도일 : 2018-11-30 00:00:00, 매도가 : 32300, 수익율 : 0.9598811292719168
매수일 : 2019-07-31 00:00:00, 매수가 : 36300
매도일 : 2019-09-30 00:00:00, 매도가 : 35150, 수익율 : 0.9683195592286501
매수일 : 2019-10-31 00:00:00, 매수가 : 33550
매도일 : 2020-03-31 00:00:00, 매도가 : 29900, 수익율 : 0.8912071535022354
매수일 : 2021-03-31 00:00:00, 매수가 : 34450
매도일 : 2021-12-30 00:00:00, 매도가 : 31500, 수익율 : 0.9143686502177069
매수일 : 2022-01-28 00:00:00, 매수가 : 30950
매도일 : 2022-02-28 00:00:00, 매도가 : 32150, 수익율 : 1.0387722132471728
매수일 : 2022-12-29 00:00:00, 매수가 : 29550
매도일 : 2023-01-31 00:00:00, 매도가 : 30350, 수익율 : 1.027072758037225
매수일 : 2023-12-28 00:00:00, 매수가 : 32350


In [23]:
df['acc_rtn'] = df['rtn'].cumprod()

In [24]:
acc_rtn = 1
for i in df.index:
    rtn = df.loc[i, 'rtn']
    acc_rtn *= rtn
acc_rtn

np.float64(0.7597942823597347)

In [26]:
df.iloc[-1, ]['종가'] / df.iloc[0, ]['종가']

np.float64(0.9017142857142857)

In [27]:
df2 = df.drop(
    ['trade', 'rtn', 'acc_rtn'], 
    axis = 1
)

In [28]:
df2['trade'] = ""

# month_last_df의 인덱스를 기준으로 반복문 실행 
for idx in month_last_df.index:
    # 절대 모멘텀 인덱스를 계산 
    momentum_index = (month_last_df.loc[idx, 'BF-1M'] / 
                    month_last_df.loc[idx, 'BF-12M']) - 1

    flag = True if (momentum_index > 0) & (momentum_index != np.inf) \
        else False

    if flag:
        # # 모멘텀 인덱스가 구매 신호 주면 해당 월말부터 
        # # 마지막 데이터까지 buy 채워준다.
        # df2.loc[idx:, 'trade'] = 'buy'
        # 모멘텀 인덱스가 구매 신호를 주면 해당 월말 다음 행의 STD-YM 추출하여
        # 해당 조건 맞는 인덱스를 필터링 하여 trade에 buy을 대입 
        std_ym = df2.shift(-1).loc[idx, 'STD-YM']
        df2.loc[std_ym, 'trade'] = 'buy'
        print(f"날짜 : {idx}, 모멘텀 인덱스 : {momentum_index}")
    

날짜 : 2016-02-29 00:00:00, 모멘텀 인덱스 : 0.014666666666666606
날짜 : 2016-06-30 00:00:00, 모멘텀 인덱스 : 0.03648648648648645
날짜 : 2016-07-29 00:00:00, 모멘텀 인덱스 : 0.07512953367875652
날짜 : 2016-08-31 00:00:00, 모멘텀 인덱스 : 0.19034482758620697
날짜 : 2016-09-30 00:00:00, 모멘텀 인덱스 : 0.2206896551724138
날짜 : 2016-10-31 00:00:00, 모멘텀 인덱스 : 0.2979274611398963
날짜 : 2016-11-30 00:00:00, 모멘텀 인덱스 : 0.532591414944356
날짜 : 2016-12-29 00:00:00, 모멘텀 인덱스 : 0.3549432739059968
날짜 : 2017-01-31 00:00:00, 모멘텀 인덱스 : 0.043363994743758294
날짜 : 2017-02-28 00:00:00, 모멘텀 인덱스 : 0.09663865546218497
날짜 : 2017-03-31 00:00:00, 모멘텀 인덱스 : 0.12483399734395739
날짜 : 2017-04-28 00:00:00, 모멘텀 인덱스 : 0.010309278350515427
날짜 : 2017-05-31 00:00:00, 모멘텀 인덱스 : 0.04041720990873543
날짜 : 2017-06-30 00:00:00, 모멘텀 인덱스 : 0.012048192771084265
날짜 : 2018-01-31 00:00:00, 모멘텀 인덱스 : 0.011494252873563315
날짜 : 2018-10-31 00:00:00, 모멘텀 인덱스 : 0.004092769440654953
날짜 : 2019-07-31 00:00:00, 모멘텀 인덱스 : 0.07909604519774005
날짜 : 2019-08-30 00:00:00, 모멘텀 인덱스 : 0.029787234

In [29]:
df2['trade'].value_counts()

trade
       1515
buy     944
Name: count, dtype: int64

In [30]:
# 수익율 계산
# 매수 일자 : 전날의 trade가 ""이고 오늘의 trade "buy"
# 매도 일자 : 전날의 trade가 "buy"이고 오늘의 trade ""

df2['rtn'] = 1

for idx in df2.index:
    # 매수의 조건식 
    if (df2.shift().loc[idx, 'trade'] == "") & \
        (df2.loc[idx, 'trade'] == "buy"):
        buy = df2.loc[idx, '종가']
        print(f"매수일 : {idx}, 매수가 : {buy}")
    # 매도의 조건식
    elif (df2.shift().loc[idx, 'trade'] == 'buy') & \
        (df2.loc[idx, 'trade'] == ""):
        sell = df2.loc[idx, '종가']
        rtn = sell / buy
        df2.loc[idx, 'rtn'] = rtn
        print(f"매도일 : {idx}, 매도가 : {sell}, 수익율 : {rtn}")

매수일 : 2016-03-02 00:00:00, 매수가 : 36250.0
매도일 : 2016-04-01 00:00:00, 매도가 : 39700.0, 수익율 : 1.0951724137931034
매수일 : 2016-07-01 00:00:00, 매수가 : 41550.0
매도일 : 2017-08-01 00:00:00, 매도가 : 40700.0, 수익율 : 0.9795427196149218
매수일 : 2018-02-01 00:00:00, 매수가 : 38300.0
매도일 : 2018-03-02 00:00:00, 매도가 : 35450.0, 수익율 : 0.9255874673629243
매수일 : 2018-11-01 00:00:00, 매수가 : 32950.0
매도일 : 2018-12-03 00:00:00, 매도가 : 32400.0, 수익율 : 0.9833080424886191
매수일 : 2019-08-01 00:00:00, 매수가 : 35800.0
매도일 : 2019-10-01 00:00:00, 매도가 : 34950.0, 수익율 : 0.9762569832402235
매수일 : 2019-11-01 00:00:00, 매수가 : 33400.0
매도일 : 2020-04-01 00:00:00, 매도가 : 28000.0, 수익율 : 0.8383233532934131
매수일 : 2021-04-01 00:00:00, 매수가 : 34150.0
매도일 : 2022-01-03 00:00:00, 매도가 : 31300.0, 수익율 : 0.9165446559297218
매수일 : 2022-02-03 00:00:00, 매수가 : 31250.0
매도일 : 2022-03-02 00:00:00, 매도가 : 32200.0, 수익율 : 1.0304
매수일 : 2023-01-02 00:00:00, 매수가 : 28900.0
매도일 : 2023-02-01 00:00:00, 매도가 : 30700.0, 수익율 : 1.0622837370242215
매수일 : 2024-01-02 00:00:00, 매수가 : 32100.0

In [31]:
df2['rtn'].cumprod()

날짜
2015-01-02    1.000000
2015-01-05    1.000000
2015-01-06    1.000000
2015-01-07    1.000000
2015-01-08    1.000000
                ...   
2024-12-24    0.801657
2024-12-26    0.801657
2024-12-27    0.801657
2024-12-30    0.801657
NaT           0.801657
Name: rtn, Length: 2459, dtype: float64

### 절대 모멘텀 함수화
1. STD-YM 컬럼을 생성하는 함수 (create_YM)
    - 매개변수 2개 
        - 데이터프레임 (_df) : 필수 
        - 기준이 되는 컬럼 : 'Adj Close' 기본값
    1. 데이터프레임 복사본 생성 
    2. 컬럼에 Date가 존재하면 index로 변경 
    3. index를 시계열로 변경 
    4. 결측치나 무한대 값들은 제거 
    5. 기준이 되는 컬럼을 제외하고 나머지는 제거 
    6. 'STD-YM' 컬럼을 생성하여 인덱스에서 년도-월 데이터를 추출하여 대입
    7. 수정이 된 데이터프레임을 되돌려준다. 

In [40]:
def create_YM(
    _df, 
    _col = '종가'
):
    df = _df.copy()
    if '날짜' in df.columns:
        df.set_index('날짜', inplace=True)
    df.index = pd.to_datetime(df.index)
    flag = df.isin( [np.nan, np.inf, -np.inf] ).any(axis=1)
    df = df.loc[~flag, ]
    df = df[[_col]]
    df['STD-YM'] = df.index.strftime('%Y-%m')
    return df

In [41]:
df = pd.read_csv("주식data/삼성카드.csv")

In [42]:
ym_df = create_YM(df)

In [43]:
ym_df

,종가,STD-YM
날짜,,
2015-01-02,43750,2015-01
2015-01-05,41300,2015-01
2015-01-06,40150,2015-01
2015-01-07,39950,2015-01
2015-01-08,40750,2015-01
...,...,...
2024-12-23,41450,2024-12
2024-12-24,40900,2024-12
2024-12-26,40950,2024-12


- 월말의 데이터프레임을 생성하는 함수 (create_last_month)
    - 매개변수 
        - STD-YM 컬럼이 생성된 데이터프레임 (_df) : 필수
        - 투자 시작시간 : 2010-01-01 기본값
        - 투자 종료시간 : 현재 시간 기본값
        - 모멘텀 기간 : 12 기본값
            - 6으로 지정한다면 전월과 6개월 전 데이터를 이용
    - 월말의 데이터만 모아서 새로운 데이터프레임을 생성 
    - 전월의 기준이되는 컬럼의 데이터를 BF1 컬럼에 대입 
    - BF2 컬럼을 생성하여 모멘텀기간만큼 전의 데이터를 대입 
    - 결측치는 0으로 대체
    - 투자의 시작 시간과 종료 시간으로 필터링 
    - 수정된 데이터프레임을 되돌려준다. 

In [44]:
def create_last_month(
    _df, 
    _start = '2010-01-01', 
    _end = datetime.now(), 
    _momentum = 12
):
    # 기준이 되는 컬럼의 이름을 변수에 저장 
    col = _df.columns[0]
    # 월말의 기준 : STD-YM이 현재와 다음행의 데이터가 다른경우
    flag = _df['STD-YM'] != _df.shift(-1)['STD-YM']
    df = _df.loc[flag, ]
    # 전월의 데이터를 생성 
    df['BF1'] = df.shift(1)[col].fillna(0)
    # _momentum의 개월 전의 데이터를 생성 
    df['BF2'] = df.shift(_momentum)[col].fillna(0)
    # 시작 시간과 종료 시간을 기준으로 필터링 
    df = df.loc[_start : _end, ]
    return df

In [45]:
month_df = create_last_month(ym_df)

- 거래 내역을 추가하는 함수 (create_rtn)
    - 매개변수 
        - create_YM() 함수의 결과 (_df1) : 필수
        - create_last_month() 함수의 결과 (_df2) : 필수
        - momentum 스코어 : 1 기본값
    1. _df1의 복사본 생성 (df 생성)
    2. df에 trade 컬럼을 생성해서 "" 대입
    3. _df2의 데이터를 이용해서 momentum_index를 생성 
    4. momentum_index 값에 따라 df 거래 내역을 추가 
    5. df에 rtn 컬럼을 생성해서 1을 대입
    6. df의 trade를 기준으로 수익율을 계산해서 rtn 컬럼에 대입 
    7. 누적 수익율을 계산하여 acc_rtn에 대입
    7. df와 최종 수익율을 되돌려준다. 

In [46]:
def create_rtn(
    _df1, _df2, 
    _score = 1
):
    df = _df1.copy()
    # trade, rtn 컬럼을 생성
    df['trade'] = ""
    df['rtn'] = 1

    # _df2를 이용해서 momentum_index를 구한다. 
    for idx in _df2.index:
        signal = ""

        # 모멘텀 인덱스를 계산 
        momentum_index = \
            _df2.loc[idx, 'BF1'] / _df2.loc[idx, 'BF2'] - _score
        
        flag = (momentum_index > 0) & (momentum_index != np.inf)

        if flag : 
            signal = 'buy'
        # df에 구매내역 추가
        df.loc[idx: , 'trade'] = signal
    # 기준이 되는 컬럼의 이름 변수에 저장 
    col = df.columns[0]

    for idx in df.index:
        # 매수 조건 
        if (df.shift().loc[idx, 'trade'] == "") & \
            (df.loc[idx, 'trade'] == "buy"):
            buy = df.loc[idx, col]
            print(f"매수일 : {idx}, 매수가 : {buy}")
        # 매도 조건
        elif (df.shift().loc[idx, 'trade'] == 'buy') & \
            (df.loc[idx, 'trade'] == ""):
            sell = df.loc[idx, col]
            rtn = sell / buy
            df.loc[idx, 'rtn'] = rtn
            print(f"매도일 : {idx}, 매도가 : {sell}, 수익율 : {rtn}")
    #  누적수익율 계산
    df['acc_rtn'] = df['rtn'].cumprod()
    acc_rtn = df.iloc[-1, -1]
    return df, acc_rtn


In [47]:
create_rtn(ym_df, month_df, 1.1)

매수일 : 2016-08-31 00:00:00, 매수가 : 44250
매도일 : 2017-01-31 00:00:00, 매도가 : 39150, 수익율 : 0.8847457627118644
매수일 : 2017-03-31 00:00:00, 매수가 : 39200
매도일 : 2017-04-28 00:00:00, 매도가 : 39900, 수익율 : 1.0178571428571428
매수일 : 2020-01-31 00:00:00, 매수가 : 37950
매도일 : 2020-03-31 00:00:00, 매도가 : 29900, 수익율 : 0.7878787878787878
매수일 : 2021-03-31 00:00:00, 매수가 : 34450
매도일 : 2021-11-30 00:00:00, 매도가 : 32450, 수익율 : 0.941944847605225
매수일 : 2023-12-28 00:00:00, 매수가 : 32350
매도일 : 2024-01-31 00:00:00, 매도가 : 33000, 수익율 : 1.0200927357032457
매수일 : 2024-03-29 00:00:00, 매수가 : 37850


(               종가   STD-YM trade  rtn   acc_rtn
 날짜                                             
 2015-01-02  43750  2015-01        1.0  1.000000
 2015-01-05  41300  2015-01        1.0  1.000000
 2015-01-06  40150  2015-01        1.0  1.000000
 2015-01-07  39950  2015-01        1.0  1.000000
 2015-01-08  40750  2015-01        1.0  1.000000
 ...           ...      ...   ...  ...       ...
 2024-12-23  41450  2024-12   buy  1.0  0.681757
 2024-12-24  40900  2024-12   buy  1.0  0.681757
 2024-12-26  40950  2024-12   buy  1.0  0.681757
 2024-12-27  39950  2024-12   buy  1.0  0.681757
 2024-12-30  39450  2024-12   buy  1.0  0.681757
 
 [2458 rows x 5 columns],
 np.float64(0.6817573957348242))